##### Copyright 2025 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Basic Code Review

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/prompting/Basic_Code_Review.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/>

Developed by [Shipra](https://github.com/Shi-pra-19).

Gemini API can be a great tool to save you time during the development process by assisting with tasks such as code review, debugging, and optimization.

Gemini API’s Python SDK can be used to perform various forms of code review tasks, including:
- Providing feedback on code quality, readability, or best practices
- Identifying weak areas in the code such as poor variable naming, inefficient loops, or security vulnerabilities
- Suggesting optimizations to improve performance and maintainability


Below is an example of using the Gemini model to enhance code quality through feedback.

In [ ]:
%pip install -U -q "google-genai>=1.7.0"

In [ ]:
from google import genai
from google.genai import types
from IPython.display import Markdown

## Configure your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

client = genai.Client(api_key=GOOGLE_API_KEY)

## Example

Start by defining some system instructions for this problem. For demonstration purposes, this use case involves prompting the model to generate a piece of code that contains a mix of common and critical mistakes.

In [ ]:
MODEL_ID="gemini-2.0-flash" # @param ["gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}

In [ ]:
buggy_code_prompt = """
     You are a beginner programmer.
     You are tasked with writing code that include common errors:

     1. Logical flaws
     2. Inefficient looping constructs
     3. Inconsistent formatting.
     4. Poor naming conventions

     Only output the code—no explanations.
     If the task doesn’t specify a programming language, default to Python.
"""

programmer_response = client.models.generate_content(
    model=MODEL_ID,
    contents= """
     Write code to take a list of integers, remove any duplicates,
     filter out all numbers less than 30,
     and return the remaining numbers sorted in descending order.
""",
    config=types.GenerateContentConfig(
        system_instruction=buggy_code_prompt
    ),
)

code = programmer_response.text
Markdown(code)

```python
def process_data(list_of_nums):
    tmp_lst = []
    for num in list_of_nums:
        if num not in tmp_lst:
            tmp_lst.append(num)

    filtered_list = []
    for i in tmp_lst:
        if i >= 30:
            filtered_list.append(i)
            
    n = len(filtered_list)
    for i in range(n):
        for j in range(0, n-i-1):
            if filtered_list[j] < filtered_list[j+1] :
                filtered_list[j], filtered_list[j+1] = filtered_list[j+1], filtered_list[j]
                
    return filtered_list
```

In [ ]:
review_prompt = """
     As a senior programmer, you are tasked with reviewing beginner-level code.
     Please follow these instructions for the evaluation:

     1. Provide detailed feedback on the code, focusing on:
        - Logic errors
        - Code readability and naming conventions
        - Code efficiency and best practices

     2. Write a corrected version of the code, addressing the identified issues.

     3. Clearly point out the changes you made and explain why they improve the original code.
"""

review = client.models.generate_content(
    model=MODEL_ID,
    contents=code,
    config=types.GenerateContentConfig(
        system_instruction=review_prompt
    ),
)


Markdown(review.text)

Okay, I'll review the provided Python code and offer feedback, a corrected version, and explanations of the improvements.

**Review of the Original Code:**

*   **Logic Errors:**  The code's logic is correct in that it aims to remove duplicates, filter numbers greater than or equal to 30, and sort the resulting list in descending order. There aren't any glaring errors that would cause the program to crash or produce completely wrong results.

*   **Code Readability and Naming Conventions:**
    *   `tmp_lst`:  While functional, `tmp_lst` is not very descriptive.  `unique_numbers` or `numbers_without_duplicates` would be better.
    *   `i`: Using `i` as a variable name for a number in `tmp_lst` is a bit confusing, as `i` is usually reserved for indices.
    *   The code is split into multiple loops, which makes it a bit harder to read. The filtering and sorting could be done in a more concise way.
    *   The bubble sort implementation is fairly standard, but it's generally better to use Python's built-in sorting functions for readability and efficiency.

*   **Code Efficiency and Best Practices:**

    *   **Duplicate Removal:**  Using `if num not in tmp_lst:` for duplicate removal is an O(n) operation within a loop, resulting in O(n^2) complexity.  Using a `set` is a much more efficient way to remove duplicates (O(n) on average).
    *   **Sorting:** Bubble sort is a simple but inefficient sorting algorithm (O(n^2)). Python's built-in `sort()` method (or `sorted()` function) uses a more efficient algorithm like Timsort (O(n log n) on average).
    *   **Filtering:** The filtering loop can be combined with the duplicate removal or done using a list comprehension for conciseness.

**Corrected Version of the Code:**

```python
def process_data(numbers):
    """
    Processes a list of numbers by removing duplicates,
    filtering numbers greater than or equal to 30,
    and sorting the result in descending order.
    """
    unique_numbers = sorted(list(set(numbers)), reverse=True)  # Remove duplicates and sort

    filtered_numbers = [num for num in unique_numbers if num >= 30]  # Filter

    return filtered_numbers
```

**Explanation of Changes and Improvements:**

1.  **Duplicate Removal with `set()`:**

    *   Original:

        ```python
        tmp_lst = []
        for num in list_of_nums:
            if num not in tmp_lst:
                tmp_lst.append(num)
        ```

    *   Improved:

        ```python
        unique_numbers = list(set(numbers))
        ```

    *   Explanation:  `set(numbers)` efficiently removes duplicates in O(n) time on average.  We then convert it back to a list using `list()`.  Using sets for duplicate removal is a standard and highly recommended practice in Python.

2.  **Combined Filtering and Sorting:**

    *   Original:

        ```python
        filtered_list = []
        for i in tmp_lst:
            if i >= 30:
                filtered_list.append(i)

        n = len(filtered_list)
        for i in range(n):
            for j in range(0, n-i-1):
                if filtered_list[j] < filtered_list[j+1] :
                    filtered_list[j], filtered_list[j+1] = filtered_list[j+1], filtered_list[j]
        ```

    *   Improved:

        ```python
        unique_numbers = sorted(list(set(numbers)), reverse=True)
        filtered_numbers = [num for num in unique_numbers if num >= 30]
        ```

    *   Explanation:  We first sort the unique numbers in descending order directly after creating the `set`. Then the filtering is done using list comprehension.

3.  **Naming Conventions:**

    *   `tmp_lst` is renamed to `unique_numbers`, and `list_of_nums` is changed to `numbers`. `filtered_list` is renamed to `filtered_numbers`. These names are more descriptive and improve readability.

4. **Docstring:**
    I added a docstring to explain what the function does. This makes the code more understandable.

5.  **Efficiency:** The corrected code is significantly more efficient because it replaces the O(n^2) duplicate removal and sorting with O(n) (average case) set creation and O(n log n) sorting. The list comprehension for filtering is also generally faster than a manual loop in Python.

In summary, the corrected code is more readable, efficient, and adheres to better Python coding practices.  It leverages built-in functions and data structures to achieve the desired functionality in a cleaner and more performant way.


## Next steps

Be sure to explore other examples of prompting in the repository. Try writing prompts for reviewing your own code as well using the example in this notebook.